In [11]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.distributions import Normal
# from torch.utils.data import DataLoader, TensorDataset

# # 使用cuda:1设备
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# # 生成示例数据：50000个2048维的向量
# data = torch.randn(50000, 2048).to(device)  # 假设数据来自标准正态分布

# # 数据加载器
# batch_size = 128
# dataset = TensorDataset(data)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Normalizing Flow的基础组件
# class RealNVP(nn.Module):
#     def __init__(self, input_dim, hidden_dim=512):
#         super(RealNVP, self).__init__()
#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim

#         # 使用简单的MLP作为网络
#         self.f = nn.Sequential(
#             nn.Linear(input_dim // 2, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, input_dim // 2)
#         )
#         self.g = nn.Sequential(
#             nn.Linear(input_dim // 2, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, input_dim // 2)
#         )

#     def forward(self, x):
#         # 划分输入向量为两部分
#         x1, x2 = x[:, :self.input_dim // 2], x[:, self.input_dim // 2:]
        
#         # 计算scale和translation
#         s = self.f(x1)
#         t = self.g(x1)
        
#         # 对x2进行仿射变换
#         z2 = (x2 - t) * torch.exp(-s)
        
#         # 返回变换后的向量以及log-determinant的雅可比行列式
#         log_det_jacobian = -s.sum(dim=1)
#         return torch.cat([x1, z2], dim=1), log_det_jacobian

# # 定义训练过程
# def train_normalizing_flow(model, dataloader, num_epochs=10, lr=1e-3):
#     optimizer = optim.Adam(model.parameters(), lr=lr)

#     model.train()
#     for epoch in range(num_epochs):
#         total_loss = 0.0
#         for batch_idx, (x,) in enumerate(dataloader):
#             x = x.to(device)
#             optimizer.zero_grad()

#             # 前向传播
#             z, log_det_jacobian = model(x)
            
#             # 使用标准正态分布计算对数似然
#             prior = Normal(torch.zeros_like(x), torch.ones_like(x))
#             log_prob = prior.log_prob(z).sum(dim=1)

#             # 负对数似然
#             loss = -(log_prob + log_det_jacobian).mean()

#             # 反向传播
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item()

#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader)}")

# # 创建模型
# model = RealNVP(input_dim=2048).to(device)

# # 开始训练
# train_normalizing_flow(model, dataloader, num_epochs=1000, lr=1e-3)


In [12]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.distributions.normal import Normal
# from torch.utils.data import DataLoader, TensorDataset
# import numpy as np

# # 定义RealNVP模型
# class RealNVP(nn.Module):
#     def __init__(self, num_dims, num_layers):
#         super(RealNVP, self).__init__()
#         self.num_dims = num_dims
#         self.num_layers = num_layers

#         # 构建RealNVP的层
#         self.made = nn.ModuleList()
#         for _ in range(num_layers):
#             self.made.append(self._build_layer())
        
#     def _build_layer(self):
#         # 定义一个RealNVP层
#         return nn.Sequential(
#             nn.Linear(self.num_dims, 512),
#             nn.ReLU(),
#             nn.Linear(512, self.num_dims * 2)  # 输出scale和translation
#         )
    
#     def forward(self, x):
#         log_det_jacobian = 0
#         for layer in self.made:
#             x, ldj = self._apply_layer(x, layer)
#             log_det_jacobian += ldj
#         return x, log_det_jacobian
    
#     def _apply_layer(self, x, layer):
#         # 分为两部分: s和t
#         s_t = layer(x)
#         s, t = s_t.chunk(2, dim=-1)
        
#         # Scale和Translation
#         z = x * torch.exp(s) + t
#         log_det_jacobian = s.sum(dim=-1)
#         return z, log_det_jacobian

# # 数据生成
# num_samples = 50000
# num_dims = 2048

# # 随机生成一些数据
# data = np.random.randn(num_samples, num_dims).astype(np.float32)
# data_tensor = torch.tensor(data)

# # 使用DataLoader
# batch_size = 256
# dataset = TensorDataset(data_tensor)
# train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # 模型、优化器和设备
# device = torch.device('cuda:1')
# model = RealNVP(num_dims=num_dims, num_layers=6).to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)

# # 训练模型
# epochs = 10
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0
#     for data in train_loader:
#         x = data[0].to(device)
#         optimizer.zero_grad()
        
#         # 正向传播
#         z, log_det_jacobian = model(x)
        
#         # 计算负对数似然
#         normal_dist = Normal(0, 1)
#         log_prob = normal_dist.log_prob(z).sum(dim=-1)
#         loss = -(log_prob + log_det_jacobian).mean()

#         # 反向传播
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
    
#     print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")

# # 测试样本
# model.eval()
# test_samples = np.random.randn(10, num_dims).astype(np.float32)
# test_tensor = torch.tensor(test_samples).to(device)

# # 计算测试样本的概率密度
# with torch.no_grad():
#     z, log_det_jacobian = model(test_tensor)
#     normal_dist = Normal(0, 1)
#     log_prob = normal_dist.log_prob(z).sum(dim=-1)
#     log_prob_density = log_prob + log_det_jacobian
#     prob_density = torch.exp(log_prob_density)

# print("Test samples probability density:", prob_density)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm 

# 设置设备为 CUDA:1
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# 定义 RealNVP 模型
class RealNVP(nn.Module):
    def __init__(self, num_features, hidden_dim=512, num_layers=8):
        super(RealNVP, self).__init__()
        
        self.num_features = num_features
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # 每一层的 scale (s) 和 translation (t) 网络
        self.s_layers = nn.ModuleList([nn.Sequential(
            nn.Linear(num_features // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_features // 2)
        ) for _ in range(num_layers)])

        self.t_layers = nn.ModuleList([nn.Sequential(
            nn.Linear(num_features // 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_features // 2)
        ) for _ in range(num_layers)])

    def forward(self, x, reverse=False):
        """
        前向或逆向变换。
        如果 reverse=True，则执行逆变换；否则执行正向变换。
        """
        x1, x2 = x.chunk(2, dim=1)  # 将输入分为两部分
        for i in range(self.num_layers):
            s = self.s_layers[i](x1)  # 缩放因子
            t = self.t_layers[i](x1)  # 平移因子
            if reverse:  # 逆变换
                x2 = (x2 - t) * torch.exp(-s)
            else:  # 正向变换
                x2 = x2 * torch.exp(s) + t
            x1, x2 = x2, x1  # 每层交换 x1 和 x2
        
        return torch.cat([x1, x2], dim=1)

    def log_prob(self, x):
        """
        计算输入 x 的对数概率密度。
        """
        x1, x2 = x.chunk(2, dim=1)
        log_det = 0  # 初始化对数雅可比行列式
        for i in range(self.num_layers):
            s = self.s_layers[i](x1)
            t = self.t_layers[i](x1)
            log_det += torch.sum(s, dim=1)  # 累积对数雅可比行列式
            x2 = (x2 - t) * torch.exp(-s)  # 更新 x2
            x1, x2 = x2, x1  # 每层交换 x1 和 x2

        z = torch.cat([x1, x2], dim=1)  # 最终的 z
        # 标准正态分布的对数概率密度
        log_prob_z = -0.5 * torch.sum(z ** 2, dim=1) - 0.5 * self.num_features * torch.log(torch.tensor(2 * torch.pi))
        return log_prob_z + log_det  # 综合对数概率

# 生成数据集
def generate_data(num_samples, num_features=2048):
    """
    生成正态分布的数据集。
    """
    return torch.randn(num_samples, num_features).to(device)

# 实例化模型并移到设备上
num_features = 2048
model = RealNVP(num_features).to(device)

# 生成训练集和测试集
train_data = generate_data(50000, num_features)
test_data = generate_data(1000, num_features)

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        # 数据的总数量
        return len(self.data)

    def __getitem__(self, idx):
        # 返回指定索引的数据
        return self.data[idx]

# 将数据加载到自定义的 Dataset 中
dataset = CustomDataset(train_data)
# 创建 DataLoader，配置 batch_size 为 256
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 使用 Adam 优化器
optimizer = optim.SGD(model.parameters(), lr=1e-5,momentum=0.9,weight_decay=0.9)

# 归一化???
# train_data = (train_data - train_data.mean(dim=0)) / train_data.std(dim=0)
# test_data = (test_data - train_data.mean(dim=0)) / train_data.std(dim=0)

# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    for x in tqdm(dataloader):
        model.train()
        optimizer.zero_grad()

        # 计算负对数似然作为损失
        loss = -torch.mean(model.log_prob(train_data))
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# 测试阶段计算测试集的概率密度
model.eval()
with torch.no_grad():
    log_prob_test = model.log_prob(test_data)  # 计算测试集对数概率
    prob_density_test = torch.exp(log_prob_test)  # 概率密度

# 打印部分测试集的概率密度结果
print("Test set probability densities (first 10):")
print(prob_density_test[:10])


 82%|████████▏ | 161/196 [01:13<00:16,  2.18it/s]